<a href="https://colab.research.google.com/github/jfernandoghe/SRANSAC/blob/master/BehaviorResearchMethods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download dataset from Dryad website [https://doi.org/10.5061/dryad.3ts56](https://datadryad.org/stash/dataset/doi:10.5061/dryad.3ts56) 

```
Drewes, Jan; Zhu, Weina; Hu, Yingzhou; Hu, Xintian (2015), Data from: Smaller is better: drift in gaze measurements due to pupil dynamics, Dryad, Dataset, https://doi.org/10.5061/dryad.3ts56
```



In [1]:
import scipy.io
import scipy
import numpy as np
import csv
from numpy import where
# !wget https://datadryad.org/bitstream/handle/10255/dryad.71532/DrewesEtAl_PlosONE_2014.tar.gz #Old File
!wget https://datadryad.org/stash/downloads/download_resource/1302__v1.zip
!ls
!unzip 1302__v1.zip
!tar -xzf DrewesEtAl_PlosONE_2014.tar.gz
mat = scipy.io.loadmat('/content/DrewesEtAl_PlosONE_2014.mat')


#                  S1 = [1600,1200]       S2 = [1920, 1080]
coorX = np.array([ [228,370,512,654,796], [470,655,840,1025,1210] ]) # S1, S2
coorY = np.array([ [100,242,384,526,668], [155,340,525,710,895]   ]) # S1, S2
# eye = 0 #Left Eye
# eye = 3 #Right Eye
nPar = 0
calGrid = 0
ligCond = 0
repe = 0
parti = 1
allData = np.zeros([0,12])
!mkdir Dataset
!mkdir OutputTechniques && cd OutputTechniques/ && mkdir L R
!mkdir OutputMetrics && cd OutputMetrics/ && mkdir L R

# Variable for all percentiles
pervar = 20

--2019-10-28 04:49:34--  https://datadryad.org/stash/downloads/download_resource/1302__v1.zip
Resolving datadryad.org (datadryad.org)... 52.89.124.62, 54.148.147.174, 52.40.64.49
Connecting to datadryad.org (datadryad.org)|52.89.124.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘1302__v1.zip’

1302__v1.zip            [ <=>                ] 159.94M  21.4MB/s    in 18s     

2019-10-28 04:49:54 (8.69 MB/s) - ‘1302__v1.zip’ saved [167711388]

1302__v1.zip  sample_data
Archive:  1302__v1.zip
  inflating: README_for_DrewesEtAl_PlosONE_2014.tar.txt  
  inflating: DrewesEtAl_PlosONE_2014.tar.gz  


## Modify dataset

In [2]:
print('Initiating...') 
for parti in range(0,39):   
  fileName = '/content/Dataset/Part'+str(parti+1)+'.csv'
  with open(fileName, 'a') as csvFile:
    writer = csv.writer(csvFile, delimiter=';') 
    print("User",parti+1, "...")
    for ligCond in range(0,7): 
      for setPoint in range(0,25):
        for repe in [0, 1, 2]:
          cX = np.array( mat['Data']['Repeat'][0][parti][0][repe][0][0][ligCond][0]['Screen_X'] [0][setPoint]  )
          cY = np.array( mat['Data']['Repeat'][0][parti][0][repe][0][0][ligCond][0]['Screen_Y'] [0][setPoint]  )
          raw = np.array(mat['Data']['Repeat'][0][parti][0][repe][0][0][ligCond][0]['Epoch']    [0][setPoint]  )
          
          
  #      OUTPUT FILE
  #      0. Participant, 1. X calpoint, 2. Y calpoint,
  #      3. X raw left eye, 4. Y raw left eye, 5. Calpoint, 6. X Timestamp, 7. Distance to left eye, 
  #      8. Repe+ligCond, 9.  X raw right eye, 10. Y raw right eye, 11. Distance to right eye
          nMat = np.column_stack(( (parti+1)*np.ones(len(raw)) , \
                                  cX[0][0]*np.ones(len(raw)),\
                                  cY[0][0]*np.ones(len(raw)),\
                                  raw[:, 1], \
                                  raw[:, 2], \
                                  ((where(coorX==cX)[1] + (5*where(coorY==cY)[1])) + 1)*np.ones(len(raw) ),\
                                  raw[:, 0], \
                                  raw[:, 3], \
                                  np.full((len(raw),1), str('D'+str(repe)+str(ligCond))), \
                                  raw[:, 4], \
                                  raw[:, 5], \
                                  raw[:, 6]))
          writer.writerows(nMat[:,:])
  #         allData = np.append(allData, nMat, axis=0)
      print("Light condition ",int(ligCond+1), "... done")
  #     break
  #   break
    csvFile.close()
  print('Done... ',ligCond+1,' light conditions for ', parti+1, ' user\n\n******')
print('*-*-*- All done -*-*-*')

!cd Dataset/ && rm Part10.csv && rm Part17.csv

Initiating...
User 1 ...
Light condition  1 ... done
Light condition  2 ... done
Light condition  3 ... done
Light condition  4 ... done
Light condition  5 ... done
Light condition  6 ... done
Light condition  7 ... done
Done...  7  light conditions for  1  user

******
User 2 ...
Light condition  1 ... done
Light condition  2 ... done
Light condition  3 ... done
Light condition  4 ... done
Light condition  5 ... done
Light condition  6 ... done
Light condition  7 ... done
Done...  7  light conditions for  2  user

******
User 3 ...
Light condition  1 ... done
Light condition  2 ... done
Light condition  3 ... done
Light condition  4 ... done
Light condition  5 ... done
Light condition  6 ... done
Light condition  7 ... done
Done...  7  light conditions for  3  user

******
User 4 ...
Light condition  1 ... done
Light condition  2 ... done
Light condition  3 ... done
Light condition  4 ... done
Light condition  5 ... done
Light condition  6 ... done
Light condition  7 ... done
Done... 

## Obtain latest modified RANSAC Regressor from public repository
Modified version of original sklearn RANSAC Regressor

In [3]:
try:
  !rm -r /content/SRANSAC
  !git clone https://github.com/jfernandoghe/SRANSAC.git   
except:
  !git clone https://github.com/jfernandoghe/SRANSAC.git       

rm: cannot remove '/content/SRANSAC': No such file or directory
Cloning into 'SRANSAC'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 35 (delta 10), reused 8 (delta 1), pack-reused 0
Unpacking objects: 100% (35/35), done.


## Define and import modules, classes, functions and packages


In [0]:

#@title
import sys
import pandas as pd
import numpy as np
import numpy.linalg as lalg
from numpy import where
import io
import re
import random
from itertools import combinations, chain
import math
from math import log
import itertools
import csv
import statistics
import scipy

def powerset(input_set, terms):
    print('terms ', terms)
    result = []
    for size in range(terms + 1):
        result += combinations(input_set, size)
    return result
      
class TipoPrueba:
  def __init__(self, pant, med, grouping): 
    self.pant = pant
    self.med = med
    self.grouping = grouping

  def __repr__(self):
    return "Ajustar columnas de mediciones %s a las de pantalla %i con agrup %i" \
  % (self.med, self.pant, self.grouping)

def modeval_ransac(A,b):
  parnum = A.ndim
  numite = 100
  AIC = 0
  AICc = 0
  MSE = 0
  return AIC, AICc, MSE

def c_pow(val, n):
  return [ val**j  for j in range(0,n+1)]
          
          
def allTerms(valXY, m, n):
  v_m = c_pow(valXY[0], m)
  v_n = c_pow(valXY[1], n)
  mmax = max(m,n)
  vals = [v_m[a]*v_n[b] for a in range(m+1) for b in range(n+1) if a+b<= mmax]
  return vals

def allSymbTerms(m, n):
  sym_m = [ f'x^{j}' if j>1 else ('x' if j == 1 else '')  for j in range(0,m+1)]
  sym_n = [ f'y^{j}' if j>1 else ('y' if j == 1 else '')  for j in range(0,n+1)]
  mmax = max(m,n)
  symb = [sym_m[a]+sym_n[b] if len(sym_m[a]+sym_n[b])>0 else '1' for a in range(m+1) for b in range(n+1) if a+b<= mmax] 
  return symb

################
#  Convert pixels to angle (in degrees)
#     parameters: 
#     v1, pixel position
#     smm, monitor size (mm)
#     d,   eye distance
def dst2Degree(v1, d, smm, spx):
  # distance to screen center (in pixels)
  v1c = v1 - spx/2 
  
  # convert pixel position in pixels to mm
  v1mm = (v1c*smm)/spx
  
  
  # calculate and return angle b 
  #      v1mm
  #     ____ 
  #     |  /
  #  d  |b/
  #     |/ 
  return math.degrees( math.atan2(v1mm, d) )



def removeSpurious(a, perc):
  num = int(round(len(a) * perc/100.0))
  for i in range(num):
    a = np.delete(a, a.argmax()) 
  return a


def removeSpuriousN(a, num):
  for i in range(num):    
    a = np.delete(a, a.argmax()) 
  return a

  

## Columns in the .csv file
0. Participant
1. X calpoint
2. Y calpoint
3. X raw left eye
4. Y raw left eye
5. Calpoint
6. X Timestamp
7. Distance to left eye
8. Repe+ligCond
9. X raw right eye
10. Y raw right eye
11.Distance to right eye





## Main algorithm

In [5]:
#
from google.colab import files
import glob
from SRANSAC import SRANSAC
# Local Variables
max_m = 3
max_n = 2
ax = ['x','y']
lc = ['0', '12.5', '25', '37.5', '50', '75', '100']
tr = ['First', 'Second', 'Third']
u_lib = ['R/','L/']
mn = (max_m+1)*(max_n+1)
max_degreeterm = 2
T = (TipoPrueba(1, (3,4), 5), TipoPrueba(1, (9,10), 5))
percData = 8
coor = [ {228,370,512,654,796,100,242,384,526,668}, {470,655,840,1025,1210,155,340,525,710,895} ] #Coordinates of calibrations points
res = [ [1600,1200], [1920, 1080] ] #Resolution in pixeles
Smm = [ [487, 274], [508, 406.4] ] #Width and Height in mm
errorFiles = []

for fname in glob.glob('/content/Dataset/Part*.csv'):
  print('file name', fname)
  df = pd.read_csv(fname, sep=';', names=['Participant', 'xp', 'yp', 'xl', 'yl', 'CalPoint', 'Timestamp', 'dL' ,'replig','xr','yr','dR'])
  medx = (3, 4)
  medx = (9, 10)

# Eliminate NaN from dataframe  
  df = df[df.xl != 0]
  df = df[df.yl != 0]
  df = df[df.xr != 0]
  df = df[df.yr != 0]
  df = df[df.xp != 0]
  df = df[df.yp != 0]
  df = df[df.CalPoint != 0]
  df = df[df.Timestamp != 0]
  df = df[df.dR != 0]
  df = df[df.dL != 0]
  df = df.fillna(0)
  df = df.values
  M = np.array(df)



  #subsampling
  M = M[::20, : ]
  
  
  part = fname[17:-4]
  print("************************************************************")
  print (part)
  print("************************************************************")
  
  for u in [0,1]: # Loop for each eye left and right      
    for e in [0,1,2]: # Loop for each try    
      for g in [5, 6]: # Loop for each light condition
        
        
        print(f'axis {u} try {e} lighting condition {g} ' )
        
        mPart = M[where(M[:, 8] == 'D'+str(e)+str(g))]
  #           mPart = M[where(M[:, 0] == part)] # Participant' data
        bb = set(mPart[:,1])     #Ejemplo de X, índice [0] 

    #***************************************
        if bool(bb & coor[0]):   #Check screentype
          mon = 0
        elif bool(bb & coor[1]):
          mon = 1    
    #***************************************


        b = mPart[:, T[u].pant]
        print(f"Shape of b {b.shape}")
        d = mPart[:, T[u].med]   # XY data

        group = mPart[:, T[u].grouping] 
        print (f' GROUP SIZE {group.size}')
        segmList = [where(group==i)[0] for i in range(1,26)]
        allTermStr = allSymbTerms (max_m, max_n)
        s1 = np.asarray( [allTerms(xy, max_m, max_n) for xy in d], dtype='float' )


        setA = {1, 17, 24, 13, 10, 19, 7, 21, 3} #Training set
        setB = {i for i in range (25) } #Total set
        setC = setB - setA #Validation set

        if ( len(set( np.unique(mPart[:,5]).flatten().astype(int)-1 ).symmetric_difference(setB))==0 ):


          segmLi1 = [segmList[i] for i in setA] # training-measurements
          merged1 = list(itertools.chain.from_iterable(segmLi1))
          segmLi2 = [segmList[i] for i in setC]   # test-measurements



          # common approach
          # (a) remove the first 10 percent of measurements (saccadic jumps SJ)
          segmLstNSJ = [ lst[math.ceil(len(lst)/percData):] for lst in segmLi1] # list without saccadic_jumps
          # (b) pick the median value
          b_mm = [ statistics.median_high(b[lst]) for lst in segmLstNSJ]
          d_mm = [ [statistics.median_high(d[lst,0]),statistics.median_high(d[lst,1])] for lst in segmLstNSJ] # XY data
          s1_mm = np.asarray( [allTerms(xy, max_m, max_n) for xy in d_mm] )


          # X-Models  0        1          2          3          4          5            6             7            8             9             10            11             12
          modelsX = [[3,0], [3, 1, 0], [4, 3, 0], [6, 3, 0], [8, 3, 0], [6, 3, 0], [4, 3, 1, 0], [8, 6, 3, 0], [1, 6, 3, 0], [4, 6, 3, 0], [1, 8, 3, 0], [2, 8, 3, 0], [4, 8, 3, 0] ]

          # Y-Models
          modelsY = [[1, 0], [3, 1, 0], [4, 1, 0], [2, 1, 0], [1, 6, 0], [1, 4, 3, 0],[2, 1, 3, 0], [5, 3, 1, 0], [4, 1, 6, 0], [7, 6, 1, 0]] 
          models =np.array((modelsX,modelsY))

          selStr = ["AIC", "AICc", "KIC", "KICc", "AICF", "KICc", "RAIC"] 


          # selectors explored: AIC, AICc, AKIC, AKICc, AICF, KICc, RAIC
          errR2_4all = np.zeros((len(models[u]), len(setA)))

          alp = 3
          de = 60
          rThreshold = 70 #(res[mon][t.grouping-5] *  (de*math.degrees(math.atan(alp))))/Smm[mon][t.grouping-5]  # Ahí le encargo!

          print(f" ========== rThreshold {rThreshold}")

          modelNum =0
          errorsEv = np.zeros([len(setC)])
          for model in models[u]: 
              maxBits = len(allTermStr)
              if len(model)>0: # and len(model)<=max_degreeterm: # Select Models that accomplish certain parameter conditions (e.g., up to 5 parameters)
                A = s1[:,model]


                ######### training phase
                ########################
                allOk = True
                try:
                  #  Fit Simple Linear model (1)
                  Am = np.array(A[merged1, :], dtype='float')
                  bm = np.array(b[merged1], dtype='float')
                  soln =  lalg.lstsq(Am, bm, rcond=None)
                  theta = soln[0]
                  residues = soln[1]


                  # Fit linear model with point selection (median)  (2)
                  A_mm = s1_mm[:,model]
                  soln_mm =  lalg.lstsq(A_mm, b_mm, rcond=None)
                  theta_mm = soln_mm[0]
                  residues_mm = soln_mm[1]


                  # Robustly fit linear model with RANSAC algorithm (Robust Linear Model) (3)
#                   ransac = SRANSAC(residual_threshold=rThreshold)
                  ransac = SRANSAC.SRANSAC(residual_threshold=rThreshold) # For SRANSAC from github
                  ransac.fitSegm(A, b, segmLi1)


                  pos = 0
                  for li in segmLi1:
                    Avalid = np.array( [A[i, :] for i in li] )
                    bvalid = np.array( [b[i] for i in li] )

                    errR = ransac.predict(Avalid)-bvalid
                    errR2 = np.multiply(errR, errR)
                    errR2_4all[modelNum][pos] =  np.percentile(errR2, pervar)
                    pos += 1


                except ValueError as e:
                  allOk = False
                  print('errX', e)




                #end_try  




                ######### evaluation phase
                ########################
                pos1 = 0
                betaR = np.zeros([len(setC)])
                betaS = np.zeros([len(setC)])
                betaC = np.zeros([len(setC)])


                # Calculate and save errors

                for li in segmLi2:
                  Atest = np.array( [A[i, :] for i in li] )
                  btest = np.array( [b[i] for i in li] )
                  disL = np.array( [(mPart[i, 7])/10 for i in li] ) 

                  # estimation from a simple model (1)
                  estimated =  np.matmul(Atest, theta)

                  # estimation from saccadic_and_median filter (2)
                  estimated_mm =  np.matmul(Atest, theta_mm)

                  # estimation from Ransac (3)
                  if allOk:    
                    estimated_rr = ransac.predict(Atest) 
                  else:
                    estimated_rr = 9999              

    #               # error in DEGREES
    #               if bool(bb & coor[0]):
    #                 mon = 0
    #               elif bool(bb & coor[1]):
    #                 mon = 1


                  ang  = [dst2Degree(btest[b], disL[b], Smm[mon][T[u].grouping-5], res[mon][T[u].grouping-5]) for b in range(len(disL))]
                  ang = np.array(ang)
                  ranAng = [dst2Degree(estimated_rr[b], disL[b], Smm[mon][T[u].grouping-5], res[mon][T[u].grouping-5]) for b in range(len(disL))]
                  comAng = [dst2Degree(estimated[b],    disL[b], Smm[mon][T[u].grouping-5], res[mon][T[u].grouping-5]) for b in range(len(disL))]
                  simAng = [dst2Degree(estimated_mm[b], disL[b], Smm[mon][T[u].grouping-5], res[mon][T[u].grouping-5]) for b in range(len(disL))]

                  betaR[pos1] = np.percentile(abs(np.array(ranAng)- ang), pervar)
                  betaS[pos1] = np.percentile(abs(np.array(comAng)- ang), pervar)
                  betaC[pos1] = np.percentile(abs(np.array(simAng)- ang), pervar)



                  pos1 += 1

                #end_for

                # Recall test  (05-2019)
                foundR = sum(x < 2 for x in betaR)
                foundC = sum(x < 2 for x in betaC)
                #print(betaR)


                usedTerms = [allTermStr[o] for o in model]

                mR = np.percentile(betaR,pervar)            
                mS = np.percentile(betaS,pervar)
                mC = np.percentile(betaC,pervar)



                print("modelNum", modelNum )
                print(usedTerms)

                print(f'Ransac:  {mR}')
                print("Simple: ", mS )
                print("Common: ", mC )
                print("-----")
                print(f'Found R(<3ang) {foundR}')
                print(f'Found C(<3ang) {foundC}')



                errorsEv[modelNum] = mR

                with open('/content/OutputTechniques/'+u_lib[u]+fname[-10:-4]+'_'+str(u)+str(e)+fname[-4:], 'a') as csvFile:
                  writer = csv.writer(csvFile, delimiter=';')

                  # participant, model, strategy, modelName, value
                  writer.writerow([part, str(modelNum), '0', usedTerms, mR])
                  writer.writerow([part, str(modelNum), '1', usedTerms, mS])
                  writer.writerow([part, str(modelNum), '2', usedTerms, mC])
                csvFile.close()      
                modelNum += 1


          #print(errR2_4all)
          ################### simulate selection
          inAll = errR2_4all > rThreshold*rThreshold*1.2*1.2   
          toDelete = min(inAll.sum(axis = 1))
          #print("points to delete", toDelete)
          bestSelectorVal = np.ones(7) *  np.inf
          selectorError = np.zeros(7)    

          modelNum = 0
          for model in models[u]: # evaluate each model      
            k = len(model)
            n = len(setA)-toDelete
            if 2*k <= n:
              rssI = removeSpuriousN(errR2_4all[modelNum], toDelete)
              #print("rssI", rssI)
              sumRsc = np.sum(rssI)
              c = rThreshold * rThreshold
              sigmaHat = sumRsc/n
              errRnorm =  rssI/sigmaHat
              rhoError = [(r*r)/2 if abs(r)<=c else c*r*r- c*c/2  for r in errRnorm]

              AIC = 2*(k) + n*math.log(sumRsc/n)
              AICc = AIC + (2*k**2 + 2*k )/ (n - k - 1) if n>k+1 else -999
              KIC = n*math.log(sumRsc/n) + 3*(k+1)
              KICc = n*math.log(sumRsc/n) + (2*(k+1)*n)/(n-k-2) - n*scipy.special.digamma((n-k)/2)*((n-k)/2) + n* math.log(n/2) if n>k+2 else -999 # definir PSI
              AKICc = n*math.log(sumRsc/n) + ((k+1)*(3*n-k-2))/(n-k-2) + k/(n-k)  if n>k and n>k+2 else -999
              AICF = n/math.log(sumRsc/n) * ((n*k)/(n-k-2) + 2) if n>k+2 else -999
              RAIC = 2*sum(rhoError) + 2*k

              selectorValues = [AIC, AICc, KIC, KICc, AICF, KICc, RAIC]
              for v in range (len(selectorValues)):
                if selectorValues[v] < bestSelectorVal[v]:
                  bestSelectorVal[v] = selectorValues[v]
                  selectorError[v] = errorsEv[modelNum]

            else:
              AIC, AICc, KIC, KICc, AKICc, AICF, RAIC = -999,-999,-999,-999,-999,-999,-999

            modelNum += 1

          #print("selectorError", selectorError)
          with open('/content/OutputMetrics/'+u_lib[u]+fname[-10:-4]+'_'+str(u)+str(e)+fname[-4:], 'a') as csvFile:
            writer = csv.writer(csvFile, delimiter=';')

            ss = 0  
            modelSe = np.where(selectorError[0]==errorsEv) if selectorError[0]!=0 else [-999]
            print("Model Selected", int(modelSe[0]))
            for mSel in selectorError:
                        # participant, model, strategy, modelName, value
              writer.writerow([part, modelSe[0], ss+1000, selStr[ss], mSel])  # participant,  selNum,  selName, value
              ss += 1
          csvFile.close()
        else:
          print("Not enough calibration points")
          errorFiles.append(fname[-10:-4])
        print('___________________________________________')
#       break
#     break
#   break

file name /content/Dataset/Part31.csv
************************************************************
Part31
************************************************************
axis 0 try 0 lighting condition 5 
Shape of b (1126,)
 GROUP SIZE 1126
 ========== rThreshold 70
n_inliers_subset 27 from (429,)
0 of 49 / 27
0 of 39 / 27
0 of 44 / 27
0 of 44 / 27
6 of 49 / 27
21 of 49 / 27
0 of 47 / 27
0 of 49 / 27
0 of 50 / 27
 =========== Score: 0.0034759868494361433
n_inliers_subset 73 from (429,)
22 of 49 / 73
10 of 39 / 73
0 of 44 / 73
0 of 44 / 73
11 of 49 / 73
6 of 49 / 73
0 of 47 / 73
24 of 49 / 73
0 of 50 / 73
 =========== Score: 0.023056690288655774
n_inliers_subset 100 from (429,)
49 of 49 / 100
1 of 39 / 100
0 of 44 / 100
0 of 44 / 100
0 of 49 / 100
0 of 49 / 100
0 of 47 / 100
42 of 49 / 100
8 of 50 / 100
 =========== Score: 1.8079305167966393
n_inliers_subset 136 from (429,)
0 of 49 / 136
10 of 39 / 136
18 of 44 / 136
3 of 44 / 136
12 of 49 / 136
19 of 49 / 136
24 of 47 / 136
11 of 49 / 136

KeyboardInterrupt: ignored

## Results in one .csv

In [0]:
!cd /content/OutputTechniques/R/ && cat *.csv > right.csv
!cd /content/OutputTechniques/R/ && mv -f right.csv /content/OutputTechniques/
!cd /content/OutputTechniques/L/ && cat *.csv > left.csv
!cd /content/OutputTechniques/L/ && mv -f left.csv /content/OutputTechniques/

## TEST CODE

In [0]:
# !rm /content/OutputMetrics/*
# !rm /content/OutputTechniques/*

In [0]:
# # X-Models  0        1          2          3          4          5            6             7            8             9             10            11             12
# modelsX = [[3,0], [3, 1, 0], [4, 3, 0], [6, 3, 0], [8, 3, 0], [6, 3, 0], [4, 3, 1, 0], [8, 6, 3, 0], [1, 6, 3, 0], [4, 6, 3, 0], [1, 8, 3, 0], [2, 8, 3, 0], [4, 8, 3, 0] ]

# # Y-Models
# modelsY = [[1, 0], [3, 1, 0], [4, 1, 0], [2, 1, 0], [1, 6, 0], [1, 4, 3, 0],[2, 1, 3, 0], [5, 3, 1, 0], [4, 1, 6, 0], [7, 6, 1, 0]] 
# models =np.array((modelsX,modelsY))
# print(models[0][0])

# Where crashes 17, 10

# With error    11, 18, 19, 35, 31, 34
# Without Error 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 36, 37, 38, 39, 02,  
print( np.unique(errorFiles) )
print(fname[-10:4])
! cd OutputMetrics/ && ls
# print(fname[-10:-4]+'_'+str(u)+str(e)+fname[-4:])